# Implicit geometries
[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meyer-nils/torch-fem/blob/main/examples/basic/solid/implicits.ipynb)

In [1]:
import torch
from torchfem import Solid
from torchfem.materials import IsotropicElasticity3D
from torchfem.mesh import cube_hexa
from torchfem.sdfs import Sphere, Cylinder, Box, Gyroid, Shell
from torchfem.io import export_mesh

import pyvista

pyvista.set_plot_theme("document")

torch.set_default_dtype(torch.float64)

# Elastic material model
material = IsotropicElasticity3D(E=1000.0, nu=0.3)

In [2]:
# Create a mesh
N = 100
nodes, elements = cube_hexa(N, N, N)

# Create a solid object
csg_sample = Solid(nodes, elements, material)

## Constructive solid geometry example
Here, we build the Constructive Solid Geometry example from [Wikipedia](https://en.wikipedia.org/wiki/Constructive_solid_geometry).

![Constructive Solid Geometry Example](https://upload.wikimedia.org/wikipedia/commons/8/8b/Csg_tree.png)


We can easily combine different objects with pythonic operators:
- Union: `|`
- Intersection: `&`
- Difference: `-`


In [3]:
S1 = Sphere(torch.tensor([0.5, 0.5, 0.5]), 0.6)
B1 = Box(torch.tensor([0.5, 0.5, 0.5]), torch.ones(3))
C1 = Cylinder(torch.tensor([0.5, 0.5, 0.5]), 0.2)
C2 = Cylinder(torch.tensor([0.5, 0.5, 0.5]), 0.2).rotate(
    torch.tensor([0.0, 0.0, 1.0]), torch.tensor(torch.pi / 2.0)
)
C3 = Cylinder(torch.tensor([0.5, 0.5, 0.5]), 0.2).rotate(
    torch.tensor([1.0, 0.0, 0.0]), torch.tensor(torch.pi / 2.0)
)

# Combine with boolean operations
body = (S1 & B1) - (C1 | C2 | C3)

csg_sample.plot(
    node_property={"SDF": body.sdf(nodes)}, contour=("SDF", [0.0]), color="skyblue"
)

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Box with infill 

In [4]:
# Create a mesh
nodes, elements = cube_hexa(100, 25, 25, 100.0, 25.0, 25.0)

# Create a solid object
model = Solid(nodes, elements, material)

# Build a gyroid and a box
center = torch.tensor([50.0, 25.0, 25.0])
G = Gyroid(center, 5 * torch.ones(3))
B = Box(center, torch.tensor([95.0, 45.0, 45.0]))
BS = Shell(B, 1.0)
GS = Shell(G, 1.0)
body = (GS & B) | BS

model.plot(
    node_property={"SDF": body.sdf(nodes)}, contour=("SDF", [0.0]), color="skyblue"
)

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [5]:
# Set constraints
model.constraints[nodes[:, 0] == 0.0, 0] = True
model.constraints[nodes[:, 1] == 25.0, 1] = True
model.constraints[nodes[:, 2] == 25.0, 2] = True
model.constraints[nodes[:, 0] == 100.0, 0] = True
model.displacements[nodes[:, 0] == 100.0, 0] = 1.0

# Create nodal density field
mask = body.sdf(nodes) > 0.0
rho_nodes = torch.ones_like(body.sdf(nodes))
rho_nodes[mask] = 0.01

# Integrate element density field
rho_elems = model.integrate_field(rho_nodes)
vol_elems = model.integrate_field(torch.ones_like(rho_nodes))
rho_elems /= vol_elems

In [6]:
# Reduce stiffness with density field
model.material.C *= rho_elems[:, None, None]

# Solve
u, f, σ, ε, α = model.solve()

In [9]:
model.plot(
    node_property={"SDF": body.sdf(nodes), "Disp": u},
    element_property={"Stress": σ, "Strain": ε},
    contour=("SDF", [0.0]),
    scalars="Stress",
    component=0,
    cmap="inferno",
    clim=[-5, 15],
)

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [8]:
export_mesh(
    model,
    "model.vtu",
    nodal_data={"SDF": body.sdf(nodes), "Disp": u},
    elem_data={"Stress": [σ], "Strain": [ε], "Density": [rho_elems]},
)